<table>
  <tr>
    <td><img src="https://github.com/rvss-australia/RVSS/blob/main/Pics/RVSS-logo-col.med.jpg?raw=1" width="400"></td>
    <td><div align="left"><font size="30">Image Features</font></div></td>
  </tr>
</table>

(c) Peter Corke 2024

Robotics, Vision & Control: Python, see Chapter 12

## Configuring the Jupyter environment
We need to import some packages to help us with linear algebra (`numpy`), graphics (`matplotlib`), and machine vision (`machinevisiontoolbox`).
If you're running locally you need to have these packages installed.  If you're running on CoLab we have to first install machinevisiontoolbox which is not preinstalled, this will be a bit slow.

In [ ]:
try:
    import google.colab
    print('Running on CoLab')
    !pip install machinevision-toolbox-python
    COLAB = True
except:
    COLAB = False

%matplotlib ipympl
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = [8, 8]

import numpy as np
import math
from machinevisiontoolbox import Image, Kernel
from spatialmath.base import plot_point


# Harris corner features
## From first principles

We now have all the ingredients (and knowledge) needed to find interest points.

In [ ]:
castle = Image.Read('castle.png', grey=True, dtype='float')
castle.disp()

The fundamental intuition behind interest points is that they have a high gradient in two orthogonal directions, but not necessarily the u- and v-axis directions.

However we start by computing the gradient in the u- and v-axis directions and then for every pixel compute this matrix

$\mathbf{A} = \begin{pmatrix} \mathbf{G}_\sigma * \mathbf{I}_u^2  & \mathbf{G}_\sigma * \mathbf{I}_u \mathbf{I}_v \\ \mathbf{G}_\sigma * \mathbf{I}_u \mathbf{I}_v & \mathbf{G}_\sigma * \mathbf{I}_v^2 \end{pmatrix}$

This $2 \times 2$ symmetric matrix is called the structure tensor.
It captures the intensity structure of the local neighborhood and its eigenvalues provide a rotationally invariant description of the neighborhood. The elements of the $\mathbf{A}$ matrix are computed from the image gradients, squared or multiplied, and then smoothed using a weighting matrix. The latter reduces noise and improves the stability and reliability of the detector.

In [ ]:
# compute derivatives

kernel = Kernel.Sobel()

Iu = castle.convolve(kernel)
Iv = castle.convolve(kernel.T)

# make a Gaussian smoothing kernel 11x11 with sigma=1
w2 = 5  # half width
k = range(-w2, w2+1)
sigma = 1
[U,V] = np.meshgrid(k, k)
kgaussian = 1.0 / (2 * math.pi * sigma**2) * np.exp(-(U**2 + V**2) / (2 * sigma**2))

# could also use kgaussian = kgauss(sigma, w2)

# compute the 3 unique elements of the structure tensor
Ix = (Iu * Iu).convolve(kgaussian)
Iy = (Iv * Iv).convolve(kgaussian)
Ixy = (Iu * Iv).convolve(kgaussian)

An interest point $(u, v)$ is one where whatever direction we move the window it rapidly becomes dissimilar to the original region. If we consider the original image $\mathbf{I}$ as a surface the eigenvalues of $\mathbf{A}$ are the principal curvatures of the surface at that point: 

* If both eigenvalues are small then the surface is flat, that is the image region has approximately constant local intensity. 
* If one eigenvalue is high and the other low, then the surface is ridge shaped which indicates an edge. 
* If both eigenvalues are high the surface is sharply peaked which we consider to be a corner.

The well known Shi-Tomasi detector considers the strength of the corner, or cornerness, as the minimum eigenvalue

$C_{\text{ST}}(u,v) = \min( \lambda_1, \lambda_2)$

where $\lambda_i$ are the eigenvalues of $\mathbf{A}$. Points in the image for which this measure is high are referred to as ["good features to track"](http://www.ai.mit.edu/courses/6.891/handouts/shi94good.pdf). 

The Harris detector is based on this same insight but defines corner strength as

$C_{H}(u,v) = \det(\mathbf{A}) - k \text{trace}(\mathbf{A})$

and again a large value represents a strong, distinct, corner. Since $\det(A) = \lambda_1  \lambda_2$ and
$\text{trace}(A) = \lambda_1 + \lambda_2$ the Harris detector responds when both eigenvalues are large and elegantly avoids computing the eigenvalues of A which has a somewhat higher computational cost.  Typically $k=0.04$.

We compute this for all pixels at once

In [ ]:
rawC = (Ix * Iy - Ixy * Ixy) - 0.04 * (Ix + Iy)**2
rawC.disp(colormap='signed');

which shows the raw value of $C_H$. If we zoom in and examine the image we will see that large areas of smooth background are around zero, edges are negative and corners of things (like the letters) have a positive value.

Next we need to find the coordinates of those positive values.  For every pixel we find the largest value in a $5\times 5$ window around each pixel but not including the centre pixel itself.

In [ ]:
# create the structuring element
SE = np.ones((5,5), np.uint8)
SE[2,2] = 0

# find the maximum value around each pixel
surrounding_max = rawC.dilate(SE)

We've done this using a morphological filtering operation called dilation, you can find more details from the documentation page for [`dilate`](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#ga4ff0f3318642c4f469d0e11f242f3b6c).

Next we find all the pixels whose value is greater than those surrounding it (which we just computed). This is a common trick in computer vision when we are looking for peaks &ndash; it's called non-local maxima suppresion.

In [ ]:
uv = (rawC > surrounding_max).nonzero()  # find the indices of all the peaks
peakvals = rawC.image[uv[1,:], uv[0,:]]  # find the peak values
k = np.argsort(-peakvals)  # sort those values into descending order
print('%d peaks found\n' % (len(k),))  # report how many peaks were found
k = k[:200]  # take first 200, these are the strongest peaks

# find the u, v coordinates as two separate lists
u = np.array([uv[0][i] for i in k])
v = np.array([uv[1][i] for i in k])

# display the image, and overlay with markers for each detected corner feature
castle.disp(darken=True, block=None);
plot_point((u, v), 'y+');

Zoom in on the letters and you will see that the features are placed on sharp corners.

### Summary

Developed by Chris Harris and Mike Stephens in 1988 the [Harris detector](https://en.wikipedia.org/wiki/Harris_Corner_Detector) was THE detector used in computer vision until
[SIFT](https://en.wikipedia.org/wiki/Scale-invariant_feature_transform) was developed by David Lowe in 1999.
SIFT is very powerful (it has the very useful property called scale-invariance which we won't have time to cover) but was [patented by the University of British Columbia](https://patents.google.com/patent/US6711293) and researchers were therefore reluctant to be too dependent on it. [SURF](https://en.wikipedia.org/wiki/Speeded_up_robust_features) was  developed by Herbert Bay, Tinne Tuytelaars, and Luc Van Gool and published in 2006. It has similar functionality to SIFT but is much faster.  It turns out that [SURF was also patented](https://worldwide.espacenet.com/patent/search/family/036954920/publication/US2009238460A1?q=pn%3DUS2009238460) but people seemed to worry less about that.  Now the SIFT patent has expired so it is back in favour.  OpenCV ships with SIFT but not SURF.

# SIFT features

In [ ]:
butterfly = Image.Read('butterfly.jpg')
butterfly.disp();

Now we can find SIFT features in this scene

In [ ]:
features = butterfly.SIFT()

which is another type of object that contains data about all the features, it behaves a bit like a list

In [ ]:
len(features)

and in this case there are 1055 features found.

Each feature (sometimes called keypoints) has a number of characteristics

In [ ]:
features[0]

which are also attributes of the object.  For example the centroid is

In [ ]:
features[0].p

Unlike the Harris feature, the SIFT feature also has a size (in pixels)

In [ ]:
features[0].scale

as well as a characteristic angle

In [ ]:
features[0].orientation

in degrees.

Each feature also strength or response (similar to "cornerness" for the Harris detector) that indicates how unique the feature is

In [ ]:
features[0].strength

The first 10 SIFT features are

In [ ]:
features[:10].table()

and they are essentially in the order they were encountered in the image.  We are often interested in the N strongest features, so we would first sort the features into descending order by strength

In [ ]:
features = features.sort()

and now the first 10 are

In [ ]:
features[:10].table()

We can plot the features, their size and orientation

In [ ]:
butterfly.disp(block=None)
features.sort(by='scale')[:50].plot(filled=True, alpha=0.5)

where each feature is denoted by a translucent disk that indicates the position and scale of the feature.  Clearly it has found the distinctive points in the image.

**Q: Compute and plot a histogram of the feature scales.**

Each feature point has a descriptor which is a summary of the pixel values within the variable size disk surrounding it. This is given by the descriptor which is a vector of 64 floats


In [ ]:
features[0].descriptor

If we moved the camera to shrink the image by a factor of two the size of all the support region disks would reduce by a factor of two, but the descriptor would remain approximately constant.  If we rotated the camera by 90degrees the characteristic angle of all the features would change by 90degrees but the feature size and the descriptor would remain approximately constant.

We can display the orientation, along with the feature size and scale by

In [ ]:
butterfly.disp(block=None)
features.sort(by='scale')[:50].plot(filled=True, alpha=0.5, hand=True)

where the blue line (like a one-handed clock) indicates the characteristic orientation.  Some features are doubled up, same position and scale, but different orientation.

# Finding correspondences between images

First we will load two different views of the same scene

In [ ]:
im1 = Image.Read('eiffel-1.png')
im2 = Image.Read('eiffel-2.png')
Image.Hstack((im1,im2)).disp()


and then use SIFT to find corresponding features in the two images

In [ ]:
f1 = im1.SIFT()
f2 = im2.SIFT()

Then we create a "*match*" object and give it the two sets of features.  It finds the best matches and for each match computes a `distance` which is a measure of dissimilarity.  We sort the matches into decreasing similarity

In [ ]:
matches = f1.match(f2)
matches

then plot the best 100 matches.  The `plot` method renders the matches onto the original images and overlays lines that connect corresponding points

In [ ]:
matches[:100].plot('y', alpha=0.6, block=None)

Zoom in and see how well it has done.

Finding corresponding points in a pair of images is critical to a lot of important computer vision problems such as stereo, structure from motion and visual SLAM.